## Do upvote, help me reach expert in notebooks :)

# Mechanisms of Action (MoA) Prediction

## Step 1: Understanding the Data

**train_features.csv** - Features for the training set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).<br>
**train_targets_scored.csv** - The binary MoA targets that are scored.<br>
**train_targets_nonscored.csv** - Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.<br>
**test_features.csv** - Features for the test data. You must predict the probability of each scored MoA for each row in the test data.<br>
**sample_submission.csv** - A submission file in the correct format.

In [ ]:
import numpy as np 
import pandas as pd
import tensorflow
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from sklearn.metrics import log_loss
import tensorflow as tf


## Loading Data

In [ ]:
data_train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
data_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
data_train_target_ns = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
data_train_target_s = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
display(data_train.head())
print("SHAPE of training data--",data_train.shape)

In [ ]:
display(data_test.head())
print("SHAPE of test data--",data_test.shape)

### Encode cp_type, cp_dose, cp_time and remove sig_id

In [ ]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72:2})
    del df['sig_id']
    return df

train = preprocess(data_train)
test = preprocess(data_test)

del data_train_target_s['sig_id']

## Creating Model 

In [ ]:
def create_model(num_columns):
    model = Sequential()
    model.add(Input(num_columns))
    model.add(BatchNormalization())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1024, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(Dense(206, activation='sigmoid'))
    
    optimizer = tfa.optimizers.Lookahead('adam',sync_period=10)
    
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    
    model.summary()
    return model

In [ ]:
def metric(y_true, y_pred):
    metrics = []
    for _target in data_train_target_s.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

## Training Model 
We will create 4 models and each model will have 5 kfold split. In last we average out the predictions and save it to csv

In [ ]:
N_STARTS = 4
tf.random.set_seed(42)

res = data_train_target_s.copy()
sub.loc[:, data_train_target_s.columns] = 0
sub.loc[:, data_train_target_s.columns] = 0

for seed in range(N_STARTS):
    for n, (train_idx, test_idx) in enumerate(KFold(n_splits=5, random_state=seed, shuffle=True).split(data_train_target_s, data_train_target_s)):
        print(f'Fold {n}')
    
        model = create_model(875)
        checkpoint_path = f'repeat:{seed}_Fold:{n}.h5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[train_idx],
                  data_train_target_s.values[train_idx],
                  validation_data=(train.values[test_idx], data_train_target_s.values[test_idx]),
                  epochs=25, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=1
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values)
        val_predict = model.predict(train.values[test_idx])
        
        sub.loc[:, data_train_target_s.columns] += test_predict
        res.loc[test_idx, data_train_target_s.columns] += val_predict
        print('')
    
sub.loc[:, data_train_target_s.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train_target_s.columns] /= N_STARTS

In [ ]:
print(f'OOF Metric: {metric(data_train_target_s, res)}')

In [ ]:
sub.loc[test['cp_type']==1, data_train_target_s.columns] = 0

In [ ]:
sub.to_csv('submission.csv', index=False)